Iteration is one of Python’s strongest features.

Python 3 uses generators in many places. Even the range() built-in now returns a generator object

Every generator is an iterator: generators fully implement the iterator interface 

Every collection in Python is *iterable*

Iterators are used internally to support:

• for loops

• Collection types construction and extension

• Looping over text files line by line

• List, dict, and set comprehensions

• Tuple unpacking

• Unpacking actual parameters with * in function calls


# Iterators and Iterables

In [1]:
items = [1, 2, 3]

To manually consume an iterable, use the `next()` function

In most cases, the *for* statement is used to consume an iterable. However, every now and then, a problem calls for more precise control over the underlying iteration mechanism.

In [2]:
it = iter(items) # invokes items.__iter__()

In [3]:
next(it)

1

In [4]:
next(it)

2

In [5]:
next(it)

3

In [6]:
next(it)

StopIteration: 

### Delegating Iteration

You have built a custom container object that internally holds a list, tuple, or some other iterable. You would like to make iteration work with your new container.

Typically, all you need to do is define an **\__iter__()** method that delegates iteration to the internally held container.

In [12]:
class Node:
    def __init__(self, value):
        self._value = value
        self._children = []
    
    def __repr__(self):
        return f'Node({self._value!r})'
    
    def add_child(self, node):
        self._children.append(node)
        
    def __iter__(self):
        return iter(self._children)

if __name__ == "__main__":
    root = Node(0)
    child1 = Node(1)
    child2 = Node(2)
    root.add_child(child1)
    root.add_child(child2)
    for i in root:
        print(i)

Node(1)
Node(2)


If `__iter__` is not implemented **TypeError: 'Node' object is not iterable** will be thrown 

In this code, the **\__iter__()** method simply forwards the iteration request to the internally held _children attribute.

Python’s iterator protocol requires **\__iter__()** to return a special iterator object that implements a **\__iter__()** method to carry out the actual iteration.

### Creating new iteration pattern

`for loop` doesn't allow to use float step size. We can implement by defining our own iterator

In [21]:
def floatrange(start, stop, step=0.5):
    x = start
    while x < stop:
        yield x
        x += step
        
for i in floatrange(0, 5):
    print(i)

0
0.5
1.0
1.5
2.0
2.5
3.0
3.5
4.0
4.5


In [25]:
list(floatrange(1,5))

[1, 1.5, 2.0, 2.5, 3.0, 3.5, 4.0, 4.5]

The mere presence of the yield statement in a function turns it into a generator. Unlike a normal function, a generator only runs in response to iteration.

In [26]:
def countdown(n):
    print(n)
    while n>=0:
        yield n
        n -= 1
    print('Done')

In [27]:
countdown(3)

<generator object countdown at 0x00000138EA6C9AC8>

No output appears. This only creates Generator object. To iterate use `next()`

In [28]:
c = countdown(3)
next(c)

3


3

In [29]:
next(c)

2

In [30]:
next(c)

1

In [31]:
next(c)

0

In [32]:
next(c)

Done


StopIteration: 

The key feature is that a generator function only runs in response to “next” operations carried out in iteration. Once a generator function returns, iteration stops.

### Iterating Reverse

In [33]:
for i in reversed([1, 2, 3, 4]):
    print(i)

4
3
2
1


Reversed iteration only works if the object in question has a size that can be determined or if the object implements a **\_\_reversed__()** special method.

**\_\_reversed__()** can be customized in user defined classes

In [34]:
class Countdown:
    def __init__(self, start):
        self.start = start
        
    # Forward iterator
    def __iter__(self):
        n = self.start
        while n > 0:
            yield n
            n -= 1
            
    # Reverse iterator
    def __reversed__(self):
        n = 1
        while n <= self.start:
            yield n
            n += 1

In [40]:
c = Countdown(3)
rev = reversed(c)

In [41]:
next(rev)

1

In [42]:
next(rev)

2

In [43]:
next(rev)

3

In [44]:
forw = iter(c)

In [45]:
next(forw)

3

In [46]:
next(forw)

2

In [47]:
next(forw)

1

In [48]:
next(forw)

StopIteration: 

### Why sequences are iterable

Whenever interpreter needs to iterate over an object x, it calls `iter(x)`

`iter()` checks whether the objects has `__iter__()` method, and calls that to iterate

If `__iter__()` is not present but `__getitem()__`  is implemented, python creates an iterator that attempts to fetch an item starting at index 0

If that fails, **TypeError** is raised

### Iterables vs Iterators

*iterable*

    Any object from which the iter built in function can obtain iterator. Objects implementing __iter__() returning iterator are iterable. 
    Sequences are always iterable so are objects implementing __getitem__ that  takes 0-based indexes
    
*iterator*

    Any object that implements the __next__ no-argument method that returns the next item in a series or raises StopIteration when there are no more items. 
    Python iterators also implement the __iter__ method so they are iterable as well.
  

In [4]:
s = 'abc'
for i in s:
    print(i, end=' ')

a b c 

If there was no `for` loop, we would need to create our own

In [8]:
it = iter(s)
while True:
    try:
        print(next(it))
    except StopIteration:
        del it
        break

a
b
c


String s i.e. 'abc' is an iterable and we built an iterator for it

**StopIteration** signals that the iterator is exhausted. This exception is handled internally in for loops and other iteration contexts like list comprehensions, tuple unpacking, etc.

The standard interface for an iterator has two methods:

**\_\_next__**
    
    Returns the next available item, raising StopIteration when there are no more items.
    
**\_\_iter__**

    Returns self; this allows iterators to be used where an iterable is expected, for example, in a for loop.

Because the only methods required of an iterator are **\_\_next__** and **\_\_iter__**

There is no way to check whether there are remaining items, other than to call `next()` and catch `StopInteration`. Also, it’s not possible to *“reset”* an iterator.

### Building a classic iterator

In [10]:
import re
import reprlib

RE_WORD = re.compile('\w+')

class Sentence:
    def __init__(self, text):
        self.text = text
        self.words = RE_WORD.findall(text)
    
    def __repr__(self):
        return f'Sentence({reprlib.repr(self.text)})'
    
    def __iter__(self):
        return SentenceIterator(self.words) #fulfils the protocol by instantiating and returning an iterator
    
class SentenceIterator:
    def __init__(self, words):
        self.words = words #reference to the list of words
        self.index = 0 #to determine next word to fetch
    
    def __next__(self):
        try:
            word = self.words[self.index] #get the word of the index
        except IndexError:
            raise StopIteration() #raise if no word
        
        self.index += 1 #increment index
        return word #return word
    
    def __iter__(self):
        return self
    
    

Note that implementing **\_\_iter__** in SentenceIterator is not actually needed for this example to work, but the it’s the right thing to do 

iterators are supposed to implement both **\_\_next__** and **\_\_iter__**, and doing so makes our iterator pass the issubclass(SentenceInterator, abc.Iterator) test.

In [20]:
s = Sentence('"The time has come,"')
it = iter(s)
print(next(it))
print(next(it))
print(next(it))
print(next(it))

The
time
has
come


Iterables have an **\_\_next__** method that instantiates a new iterator every time.

Iterators implement a **\_\_next__** method that returns individual items, and an **\_\_iter__** method that returns self.

`**Therefore, iterators are also iterable, but iterables are not iterators.**`

An iterable should never act as an iterator over itself. In other words, iterables must implement **\_\_iter__**, but not **\_\_next__**.

On the other hand, for convenience, iterators should be iterable. An iterator’s **\_\_iter__** should just return self.

# Generators